In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install rich

In [3]:
import sys
sys.path.append("../src/agents")

from rich.console import Console
console = Console()

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
# Fix import error by explicitly importing ChatHistoryAgentThread
from semantic_kernel.contents import ChatHistory, ChatMessageContent
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Order Processing Test with Markdown Table Summary

This notebook tests the order processing flow with all agents producing a consistently updated markdown table summary that tracks changes through the order lifecycle.

In [ ]:
# Function to update agent instructions to include markdown table output
def update_agent_instructions(agent, template_section):
    # Extract original instructions
    original_instructions = agent.instructions
    
    # Add markdown table requirements
    table_instructions = f"""
{template_section}

## ORDER SUMMARY TABLE
You MUST include a markdown table at the end of your response that summarizes the order status.
This table serves as a consistent tracking mechanism as the order moves through the system.
DO NOT remove any columns added by previous agents - you may only add new columns or modify values.
Format your table as follows:

```markdown
### Order Summary Table

| SKU | Description | Original Quantity | Available Quantity | Status | [Your additional columns] |
| --- | ----------- | ----------------- | ------------------ | ------ | ------------------------ |
| SKU-A100 | Sport T shirt | 800 | ? | ? | ? |
| SKU-A102 | Hoodie | 20 | ? | ? | ? |
| ... | ... | ... | ... | ... | ... |
```

The table must include ALL items in the order. If you modify any values or add new columns, clearly mark
these changes in your own agent-specific column. This table will be used by subsequent agents to track
the order throughout its processing lifecycle.
"""
    
    # Update the agent instructions by adding the table section
    # We insert it before the last section which typically contains the corner cases
    parts = original_instructions.split("## CORNER CASES")
    
    if len(parts) > 1:
        new_instructions = parts[0] + table_instructions + "\n\n## CORNER CASES" + parts[1]
    else:
        # If no CORNER CASES section, add to the end
        new_instructions = original_instructions + "\n\n" + table_instructions
        
    # Update the agent's instructions
    agent.instructions = new_instructions
    
    return agent

In [ ]:
# Import the agents and update their instructions
from order.validator_agent import validator_agent
from order.substitution_agent import substitution_agent
from order.price_agent import pricing_agent
from order.fulfillment_agent import fulfillment_agent

# Update validator agent instructions
validator_agent = update_agent_instructions(validator_agent, 
    "Your validation report must include a markdown table summary of the order status.")

# Update substitution agent instructions
substitution_agent = update_agent_instructions(substitution_agent,
    "Your substitution analysis must include the updated markdown table showing which items were substituted.")

# Update pricing agent instructions
pricing_agent = update_agent_instructions(pricing_agent,
    "Your pricing report must include the updated markdown table showing pricing details for all items.")

# Update fulfillment agent instructions
fulfillment_agent = update_agent_instructions(fulfillment_agent,
    "Your fulfillment report must include the final markdown table showing delivery status for all items.")

In [ ]:
# Update the order_team.py file to use the modified agents
from order.order_team import processing_team

In [ ]:
history = ChatHistory()

# Read order from file in /data
with open("data/input_order_1.json", "r") as file:
    order = file.read()

history.add_user_message(f"Please process the following order with detailed markdown table summaries after each step: {order}")

async for response in processing_team.invoke(history=history):
    msg: ChatMessageContent = response
    logger.info(f"{msg.name}: {msg.content}")

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.21.0 Python/3.13.2 (Windows-11-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> retry_mechanism=PassThroughWithoutRetry() services={'o3-mini': AzureChatCompletion(ai_model_id='o3-mini', service_id='o3-mini', instruction_role='system', client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001E27F7EF390>, ai_model_type=<OpenAIModelTypes.CHAT: 'chat'>, prompt_tokens=0, completion_tokens=0, total_tokens=0)} ai_service_selector=<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x000001E210953680> plugins={} function_invocation_filters=[] prompt_rendering_filters=[] auto_function_invocation_filters=[]


INFO:httpx:HTTP Request: POST https://oai-tst-sweden-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=1113, prompt_tokens=1292, total_tokens=2405, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=576, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=1024))
INFO:semantic_kernel.functions.kernel_function:Function CreatePlan succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 9.608322s
INFO:sk_ext.planning_strategy:CreatePlan: {
    "plan": [
        {
            "agent_id": "validator_agent",
            "instructions": "Step 1: Validate the entire order. Use the 'validate_skus' tool to ensure all SKUs (SKU-A100, SKU-A102, SKU-A103, SKU-A104, SKU-A114) exist i

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

...

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Please process the following order: {
  "customerId": "cust001",
  "customerName": "Acme Corporation",
  "order": [
    {
      "sku": "SKU-A100",
      "description": "Sport T shirt",
      "size": "M",
      "color": "Red",
      "quantity": 800,
      "unit_price": 10.0
    },
    {
      "sku": "SKU-A102",
      "description": "Hoodie",
      "size": "M",
      "color": "Black",
      "quantity": 20,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A103",
      "description": "Hoodie",
      "size": "L",
      "color": "Black",
      "quantity": 30,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A104",
      "description": "Elephant T-shirt",
      "size": "S",
      "color": "White",
      "quantity": 5,
      "unit_price": 100.0
    },
    {
      "sku": "SKU-A114",
      "description": "Zebra T-shirt",
      "size": "XXL",
      "color": "White",
      "quantity": 5,
      "unit_price": 100.0
    }
  ]
}

----------------------------------------------------------------------------------------------------

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

INFO:httpx:HTTP Request: POST https://oai-tst-sweden-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=3676, prompt_tokens=6166, total_tokens=9842, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=2688, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.connectors.ai.chat_completion_client_base:processing 1 tool calls in parallel.
INFO:semantic_kernel.kernel:Calling FulfillmentPlugin-finalize_order function with args: {"order_id": "order-20250405-000001", "updated_order": {
  "customerId": "cust001",
  "customerName": "Acme Corporation",
  "order_id": "order-20250405-000001",
  "line_items": [
    {
      "sku": "SKU-A100",
      "description": "Sport T-Shirt",
      "size

In [ ]:
# Extract and display all markdown tables from the conversation history
import re

def extract_markdown_tables(text):
    # Regex to find markdown tables with headers and at least one row
    pattern = r"(\|[^\|]+\|[^\|]+\|.*\n\|\s*[-:]+\s*\|\s*[-:]+\s*\|.*\n(\|.*\|.*\n)+)"
    tables = re.findall(pattern, text)
    return [table[0] for table in tables]

for i, msg in enumerate(history.messages):
    if msg.content and msg.role == 'assistant':
        tables = extract_markdown_tables(msg.content)
        if tables:
            print(f"\n--- {msg.name} ORDER SUMMARY TABLE ---")
            for table in tables:
                print(table)

## Analysis of Order Processing Tables

The tables above show how each agent builds upon the work of previous agents, preserving the order history while adding their own specialized information. This demonstrates the complete audit trail of order processing.

In [7]:
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()

In [8]:
for m in history.messages:
    print(f"{m.name}: {m.content}")

None: Please process the following order: {
  "customerId": "cust001",
  "customerName": "Acme Corporation",
  "order": [
    {
      "sku": "SKU-A100",
      "description": "Sport T shirt",
      "size": "M",
      "color": "Red",
      "quantity": 800,
      "unit_price": 10.0
    },
    {
      "sku": "SKU-A102",
      "description": "Hoodie",
      "size": "M",
      "color": "Black",
      "quantity": 20,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A103",
      "description": "Hoodie",
      "size": "L",
      "color": "Black",
      "quantity": 30,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A104",
      "description": "Elephant T-shirt",
      "size": "S",
      "color": "White",
      "quantity": 5,
      "unit_price": 100.0
    },
    {
      "sku": "SKU-A114",
      "description": "Zebra T-shirt",
      "size": "XXL",
      "color": "White",
      "quantity": 5,
      "unit_price": 100.0
    }
  ]
}

order_processing_team: Step 1: Validate the entire o

In [9]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions import kernel_function
from utils.config import get_azure_openai_client
from utils.store import get_data_store
import datetime


sku_list =  ["SKU-A100", "SKU-A102", "SKU-A103", "SKU-A104"]

data_store = get_data_store()

avaiilable_skus = await data_store.query_data("SELECT * FROM c", "sku")
skus_dict = {sku["id"]: sku for sku in avaiilable_skus}

# Check if all SKUs are available
invalid_skus = [sku for sku in sku_list if sku not in skus_dict]
invalid_skus


[]

In [10]:
results = {}

order = [
    {
      "sku": "SKU-A100",
      "description": "Sport T shirt",
      "size": "M",
      "color": "Red",
      "quantity": 4,
      "unit_price": 10.0
    },
    {
      "sku": "SKU-A102",
      "description": "Hoodie",
      "size": "M",
      "color": "Black",
      "quantity": 20,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A103",
      "description": "Hoodie",
      "size": "L",
      "color": "Black",
      "quantity": 30,
      "unit_price": 25.0
    },
    {
      "sku": "SKU-A104",
      "description": "Elephant T-shirt",
      "size": "S",
      "color": "White",
      "quantity": 5,
      "unit_price": 100.0
    }
  ]

    
facilities = await data_store.query_data(
      f"SELECT * FROM c", 
      "facility"
  )

console.print(f"Facilities for SKU {sku_id}: {facilities}")

for item in order:
    sku_id = item["sku"]
    quantity = item["quantity"]

    
    
    total_available = 0
    locations = []
    
    for facility in facilities:
        for sku_availability in facility.get("skuAvailability", []):
          if sku_id in sku_availability["sku"]:
              available = sku_availability["availableQuantity"]
              total_available += available
              locations.append({
                  "facility_id": facility["id"],
                  "name": facility.get("name", "Unknown"),
                  "available": available
              })
      
    results[sku_id] = {
        "requested": quantity,
        "available": total_available,
        "is_available": total_available >= quantity,
        "locations": locations
    }

results



NameError: name 'sku_id' is not defined

In [ ]:
from order.validator_agent import validator_agent

In [ ]:
validator_agent.kernel.__dict__

In [ ]:
# Save the modified agent instructions for future use
def save_agent_instructions():
    # Create a dictionary with agent instructions
    instructions = {
        "validator_agent": validator_agent.instructions,
        "substitution_agent": substitution_agent.instructions,
        "pricing_agent": pricing_agent.instructions,
        "fulfillment_agent": fulfillment_agent.instructions
    }
    
    import json
    import os
    
    # Create directory if it doesn't exist
    os.makedirs("./output", exist_ok=True)
    
    # Save instructions to file
    with open("./output/agent_instructions.json", "w") as f:
        json.dump(instructions, f, indent=2)
    
    print("Agent instructions saved to ./output/agent_instructions.json")

# Uncomment to save instructions
# save_agent_instructions()